In [ ]:
import numpy as np
import glob

import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

import matplotlib.pyplot as plt

import segmentation_models as sm
sm.set_framework('tf.keras')

In [ ]:
BACKBONE = 'resnet18'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
seed = 111
batch_size = 8
img_size = (224, 224)

datagen_args = dict(
    rotation_range=5,
    shear_range=10,
    zoom_range=[1, 1.2],
    vertical_flip=True,
    horizontal_flip=True
)

image_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    preprocessing_function=preprocess_input, 
    validation_split=0.2,
    **datagen_args,
)

mask_datagen = keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    **datagen_args
)

image_generator = image_datagen.flow_from_directory(
    './PaintDetectionDataset/train/images/', 
    class_mode=None, 
    seed=seed,
    target_size=img_size,
    batch_size=batch_size,
    subset='training'
)

mask_generator = mask_datagen.flow_from_directory(
    './PaintDetectionDataset/train/masks/',
    class_mode=None, 
    color_mode='grayscale',
    seed=seed,
    target_size=img_size,
    batch_size=batch_size,
    subset='training'
)

train_generator = zip(image_generator, mask_generator)

In [ ]:
image_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    preprocessing_function=preprocess_input, 
    validation_split=0.2
)
mask_datagen = keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2
)

val_image_generator = image_datagen.flow_from_directory(
    './PaintDetectionDataset/train/images/', 
    class_mode=None, 
    seed=seed,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation'
)
val_mask_generator = mask_datagen.flow_from_directory(
    './PaintDetectionDataset/train/masks/',
    class_mode=None, 
    color_mode='grayscale',
    seed=seed,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation'
)

val_generator = zip(val_image_generator, val_mask_generator)

In [ ]:
fig, axs = plt.subplots(2, 12, figsize=(32, 4))
for i in range(12):
    x = next(train_generator)
    axs[0, i].imshow(x[0][0])
    axs[1, i].imshow(x[1][0])

In [ ]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)
model.summary()

In [ ]:
import math
def spe(y, batch_size):
    return int(math.ceil((1. * y) / batch_size))

In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(1e-5), loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])

callbacks = [
    keras.callbacks.ModelCheckpoint("painting_segmentation.h5", save_best_only=True)
]

epochs = 200

history = model.fit(
    train_generator, 
    epochs=epochs, 
    steps_per_epoch=spe(186, batch_size), 
    validation_data=val_generator,
    validation_steps=spe(46, batch_size),
    callbacks=callbacks
)

In [ ]:
def plot_history(history, save=False, filepath=None):
    x_plot = list(range(1, len(history["loss"]) + 1))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.plot(x_plot, history["loss"])
    ax1.plot(x_plot, history["val_loss"])
    ax1.legend(["Training", "Validation"])

    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("IOU")
    ax2.plot(x_plot, history["iou_score"])
    ax2.plot(x_plot, history["val_iou_score"])
    ax2.legend(["Training", "Validation"], loc="lower right")

    fig.show()

    if save and filepath is not None:
        fig.savefig(filepath)

In [ ]:
plot_history(history.history)

In [ ]:
testdir = './PaintDetectionDataset/test/images/img/*.jpg'

fig, axes = plt.subplots(5, 10, sharex=True, sharey=True, figsize=(25, 8))

paths = glob.glob(testdir)

images = np.zeros((len(paths), 224, 224, 3))
for i, img_path in enumerate(paths):
    img = load_img(img_path, target_size=(224, 224))
    img = img_to_array(img) / 255.0
    img = preprocess_input(img)
    images[i] = img

preds = model.predict(images)

for i in range(len(paths)):
    mask = preds[i]
    mask = (mask >= 0.5).astype(np.uint8)

    k = i // 10
    j = i % 10

    axes[k, j].imshow(images[i])
    axes[k, j].imshow(mask, 'jet', interpolation='none', alpha=0.7)
    axes[k, j].axis('off')

fig.tight_layout()